<a id="orbit-project-top"></a>
<div style="width: 50px; height: 50px; border-radius: 50%; background: #2E86C1; margin-bottom: 0px; text-align: center;">
    <img src="../img/Welcome Page/orbit.png" style="max-width: 150%; max-height: 150%; right: 25%; bottom: 13px; position: relative;">
</div>
<h1 style="margin-top: 20px; margin-bottom: 5px;">Orbit: Final Project<br><span style="margin-bottom: 0px; color: #2E86C1;">Build and Publish an Interactive Dashboard</span></h1>

---
### Project Assignment:
   - Use BQL to retrieve 5 fields of current cross-sectional data (e.g. Yield, Price, Issuer, Credit Rating, etc.) for a bond universe of your choosing
   - Display the data using a DataGrid
   - Use Widgets of your choosing to filter the DataGrid
   
<h5>Example Output:</h5>
<img src="../img/Orbit/orbit_project2.gif" style="max-height: 300px;">

### Remember the 5-Step Process for Building Apps in BQuant:
   1. Setup the Environment
   2. Get Data with BQL
   3. Additional Data Munging, if necessary
   4. Build UI and Visualizations
   5. Simplify & Document
   

See if you can solve this problem in the empty Python cell below. We've written one possible solution. Try it out for yourself first, and when you're ready to see the answer, click the <img src="../img/Controls/hidden_cell.png"> icon below to expand a hidden Python cell. Try not to peek too early! We all learn best by overcoming obstacles, so if you get stuck, reference the previous material in this section before expanding the answer. Always remember that in programming there are multiple ways to accomplish the same task. If your code doesn't look exactly like ours, that's ok. The important thing is that your code executes as expected.

In [2]:
# your code here



<p style="color: orange; font-size: 12px;">&darr; Click below to expand answer</p>

In [ ]:
# # hide this code
# # Step 1: Setup Environment
import bql
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ipydatagrid import DataGrid, TextRenderer
# from numpy import nan
bq = bql.Service()

# Step 2: Get Data with BQL
active_bonds = bq.univ.bondsuniv('Active')
amount_out = bq.data.amt_outstanding(Currency='USD') > 10*10**9
street_convention = bq.data.calc_typ() == '1'
filtered_bonds = active_bonds.filter(amount_out).filter(street_convention)

issuer_name = bq.data.issuer()
country = bq.data.country_full_name()
price = bq.data.px_last(fill='prev')
bond_yield = bq.data.yield_(fill='prev')
callable_bond = bq.data.callable()
maturity = bq.data.maturity()
coupon = bq.data.cpn()
req = bql.Request(filtered_bonds, {'Issuer': issuer_name,
                                   'Country': country,
                                   'Last Price': price,
                                   'Coupon': coupon,
                                   'Yield': bond_yield,
                                   'Callable': callable_bond,
                                   'Maturity': maturity})
res = bq.execute(req)
df = pd.concat([field.df()[field.name] for field in res], axis=1)

# Skip Step 3 - no additional Data Munging Necessary
# Step 4 - build Visualizations
date_format = TextRenderer(format='%Y-%m-%d', 
                           format_type='time', 
                           horizontal_alignment='center')
num_format = TextRenderer(format='.2f', 
                          horizontal_alignment='center')
grid = DataGrid(dataframe=df,
                base_column_size=100,
                column_widths={'ID': 100,
                               'Issuer': 200},
                layout={'height': '300px'},
                renderers={'Last Price': num_format,
                           'Maturity': date_format,
                           'Yield': num_format})
def detect_nan(values_list):
    """Return true if nan is in a list of values"""
    return True in [str(i).lower() == 'nan' for i in values_list]

def get_slider_options(column_name):
    """Return a dictionary of slider options based on DataFrame column values"""
    values = df[column_name].tolist()
    if detect_nan(values):
        values = [i for i in values if str(i).lower() != 'nan']
    
    if column_name == 'Maturity':
        max_date = max(values) + pd.Timedelta('31 days')
        min_date = min(values) - pd.Timedelta('31 days')
        slider_values = pd.date_range(min_date, max_date, freq='M')
        labels = [i.strftime(' %Y-%m-%d ') for i in slider_values]
    else:
        slider_values = sorted(list(set(values)))
        slider_values.append(max(slider_values) + 0.01)
        slider_values.insert(0, min(slider_values) - 0.01)
        slider_values = [round(i, 3) for i in slider_values]
        labels = [f'{i:.2f}' for i in slider_values]
    options = list(zip(labels, slider_values))
    return options


    
def create_slider(column_name):
    """Create a slider widget for a column"""
    options = get_slider_options(column_name)
    slider = widgets.SelectionRangeSlider(
        options=options,
        index=(0, len(options) - 1),
        description=column_name,
        orientation='horizontal',
        layout={'width': '400px'}
    )
    return slider


# create sliders on 4 columns
slider_columns = ['Maturity', 'Yield', 'Coupon', 'Last Price']
sliders = dict(zip(slider_columns, [create_slider(col) for col in slider_columns]))

def get_filter_bounds():
    """get upper and lower values for all sliders"""
    filter_bounds = {}
    for column, slider in sliders.items():
        upper_bound = slider.value[1]
        lower_bound = slider.value[0]
        filter_bounds[column] = [lower_bound, upper_bound]
    return filter_bounds

def slide_filter(evt):
    """Filter the grid based on slider values"""
    slider_bounds = get_filter_bounds()
    index_levels = df.index.nlevels
    transforms = []
    for column, bounds in slider_bounds.items():
        col_index = list(df.columns).index(column) + index_levels
        transforms.append({'type': 'filter',
                           'columnIndex': col_index,
                           'operator': 'between',
                           'value': bounds})
    grid.transform(transforms)
    

# apply the callback function to each slider
for slider in sliders.values():
    slider.observe(slide_filter, 'value')
    

# build slider box UI
sliders_box_left = widgets.VBox(list(sliders.values())[:2])
sliders_box_right = widgets.VBox(list(sliders.values())[2:])
slider_box = widgets.HBox([sliders_box_left, sliders_box_right])

# display UI
display(widgets.HTML('<h1>Interactive Bond Screener</h1>'))
display(slider_box)
display(grid)
    


    

----
<p style="text-align:center;">
    Click on the links below to continue learning.<br>
    <a href="3 Debugging.ipynb">&larr; Back to Debugging & Maintenance</a>&emsp;&emsp;
    <a href="#orbit-project-top">&uarr; Return to Top</a>&emsp;&emsp;
    <br>
    <br>
    <a href="../Welcome.ipynb#welcome-top" style="font-size: 12px;">Return to the Welcome Page</a>
</p>